# Imports

In [1]:
import sys
sys.path.insert(0, '..')
import os
import time
from dotenv import load_dotenv

import math
import pandas as pd
import requests
from tqdm import tqdm
from pydantic import BaseModel
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, List, Callable, Tuple
from langchain_openai import ChatOpenAI


from config import stances
from DB_connection import DB_connection
from llm_labeling import VideoClassification, BatchClassification, build_prompt, query_llm, get_langchain_llm


load_dotenv()


OPEN_ROUTER_API_KEY = os.getenv("OPEN_ROUTER_API_KEY")
db_connection = DB_connection()

# 1. Functions

## 1.1. Create Tables

In [2]:
def create_compare_video_annotations_table():
    try:
        query = """
                SELECT * FROM compare_video_annotations
                """
        db_connection.select(query)
    except:
        query = """
                SELECT video_youtube_id,title,description,channel_title,transcript FROM annoted_videos 
                """
        videos = db_connection.select(query)
        videos = videos.sample(n=380, random_state=42).reset_index(drop=True)
        videos['true_label'] = None
        videos = videos.rename(columns={
                "channel_title": "channel_name"})
        
        

        annotated_videos = pd.read_csv('../data/annotated_videos.csv')
        annotated_videos = annotated_videos.rename(columns={
                "video_id":"video_youtube_id",
                "stance": "true_label",
                "transcript_text": "transcript",
        })
        annotated_videos = annotated_videos.drop(columns="stance_code")

        all_columns = set(videos.columns).union(set(annotated_videos.columns))
        videos = videos.reindex(columns=all_columns)
        annotated_videos = annotated_videos.reindex(columns=all_columns)

        all_videos = pd.concat([videos, annotated_videos], ignore_index=True)
        column_order = ["video_youtube_id", "title", "description", "channel_name", "transcript", "true_label"]
        all_videos = all_videos.reindex(columns=column_order)

        db_connection.save_df(all_videos, 'compare_video_annotations')
 

def create_compare_llms_tables():
    try:
        query = """
                SELECT * FROM llm_comparison
                """
        db_connection.select(query)
    except:
        query = """ CREATE TABLE llm_comparison(
                llm_name TEXT,
                llm_full_name TEXT,
                batch_size INT,
                accuracy FLOAT,
                precision FLOAT,
                recall FLOAT,
                running_time FLOAT,
                cost FLOAT
                )"""
        db_connection.create_table(query)

    
create_compare_video_annotations_table()
create_compare_llms_tables()

    


## 1.2. Open router costs

In [3]:
def get_usage_openrouter():
    resp = requests.get(
        "https://openrouter.ai/api/v1/credits",
        headers={"Authorization": f"Bearer {OPEN_ROUTER_API_KEY}"},
        timeout=30,
    )
    data = resp.json()["data"]
    return data["total_usage"]


## 1.3. LLM annotation

In [ ]:
def ensure_unique_column( base_name: str) -> str:
    # Get existing columns
    existing = set(db_connection.select("PRAGMA table_info(compare_video_annotations);")["name"])
    if base_name not in existing:
        col = base_name
    else:
        # Find smallest suffix not yet used
        k = 1
        while f"{base_name}_{k}" in existing:
            k += 1
        col = f"{base_name}_{k}"
    # Add the column safely (quote the identifier)
    db_connection.execute_query(f'ALTER TABLE compare_video_annotations ADD COLUMN "{col}" TEXT')
    return col

def annotate_by_batch(llm_structured, column_name, batch_size=10):
    
    videos = db_connection.select("SELECT * FROM compare_video_annotations")
    

    for i in tqdm(range(batch_size, len(videos), batch_size)):
        batch = videos.iloc[i:i+batch_size]
        #print(batch)
        try:
            prompt = build_prompt(batch,True)
            response = query_llm(prompt,llm_structured)
        except:
            prompt = build_prompt(batch,False)
            response =query_llm(prompt,llm_structured)
        
        for item in response.items:
            #print(f"Video ID: {item.video_id}, Label: {item.label}")

            query = f"""
                UPDATE compare_video_annotations
                SET {column_name} = ? 
                WHERE video_youtube_id = ?;
            """    
            cursor = db_connection.connection.cursor()
            cursor.execute(query,(item.label, item.video_id))
        db_connection.connection.commit()


def get_accuracy(column_name):
    query = f""" SELECT * 
                 FROM compare_video_annotations 
                 WHERE true_label IS NOT NULL 
                 AND {column_name} IS NOT 'Undefined';"""
    df = db_connection.select(query)

    correct = (df[column_name] == df["true_label"]).sum()
    total = len(df)
    accuracy = correct / total

    return accuracy

    

def label_with_one_llm(full_model_name, column_name, batch_size=10):
    column_name = column_name +f"_batch_size_{batch_size}"
    column_name = ensure_unique_column(column_name)

    start_usage = get_usage_openrouter()

    start_time = time.time()
    llm = get_langchain_llm("openrouter", full_model_name)
    llm_structured = llm.with_structured_output(BatchClassification)
    annotate_by_batch(llm_structured,column_name,batch_size)
    end_time = time.time()
    running_time = end_time - start_time

    end_usage = get_usage_openrouter()
    cost = (end_usage - start_usage) 

    accuracy = get_accuracy(column_name)
    print(f"Accuracy for {full_model_name}: {accuracy:.2f}")
    query = f"""
            INSERT INTO llm_comparison (llm_name, llm_full_name, batch_size, accuracy, running_time, cost)
            VALUES (?, ?, ?, ?, ?, ?)
            """
    cursor = db_connection.connection.cursor()
    cursor.execute(query, (column_name, full_model_name, batch_size, accuracy, running_time, cost))
    db_connection.connection.commit()


    
label_with_one_llm("mistralai/mistral-small-3.2-24b-instruct","mistralai_small",10)

    
   


100%|██████████| 50/50 [07:32<00:00,  9.05s/it]

Accuracy for mistralai/mistral-small-3.2-24b-instruct: 0.87


In [45]:
print(list(db_connection.select("PRAGMA table_info(compare_video_annotations);")["name"]))

['video_id', 'title', 'description', 'channel_name', 'transcript', 'true_label']
